In [ ]:
import starterkits.starterkit_4_2.support as sp
import starterkits.starterkit_4_2.visualizations as vis
import pandas as pd
pd.options.display.max_columns = 25

from pathlib import Path
DATA_PATH = Path('../../data/')

%load_ext autoreload
%autoreload 2

# Starter Kit 4.2: FedRepo: mitigate concept drift in federated context

## Passport

### Business context
Forecasting electricity consumption accurately is crucial for managing energy resources efficiently, ensuring reliable power supply, and optimizing grid operations. This task is complex, particularly because of the diverse and evolving usage patterns across different households. Usage patterns vary not only across households, but also over time (e.g. number of occupants changes). This leads to what is known as concept drift, which occurs when the underlying data patterns that predictive models have learned change, potentially degrading the model’s performance.

In scenarios where data privacy is paramount and bandwidth is limited, the concept of federated learning emerges as a vital approach. Federated learning allows multiple decentralized entities—such as different households in the case of electricity consumption—to collaboratively learn a predictive model while keeping all training data local. This method avoids the need to transfer large volumes of sensitive data to a central server, addressing significant privacy concerns. However, federated learning environments face significant challenges among which concept drift is a prominent one:

 - Dynamic and heterogeneous data sources often lead to concept drift, where the underlying data patterns the model has learned change over time, potentially degrading the model's performance.
 - Limited bandwidth for communication between clients and the central server can hinder the efficiency of model updates and retraining processes.
 - Privacy concerns limit the amount and type of data that can be shared between clients, complicating the detection and mitigation of concept drift.

These challenges necessitate advanced strategies for model training and maintenance to ensure that predictive models remain accurate and efficient over time without compromising privacy or incurring prohibitive communication costs.

### Business goal

The business goal for this Starter Kit is **electricity forecasting** in a collaborative distributed environment, leveraging the knowledge at each individual entity through federated learning. Specifically, a **concept drift mitigation** strategy will be applied to cope with the occurrence of concept drift due to the heterogeneous and dynamic nature of electricity usage data. The methodology, called *FedRepo*, was introduced by Tsiporkova et al. [1] and aims to provide a robust solution that maintains the accuracy and efficiency of the federated models over time, ensuring they adapt to changes in data dynamics while minimizing communication overhead.

### Application context
The FedRepo methodology is applicable in various settings where data privacy and limited connectivity are major concerns:

 - Healthcare: Hospitals and medical institutions can collaborate on developing predictive models or improving diagnostic tools on more diverse data without actually sharing patient data. 
 - Wearables: User experience of several features (e.g. text prediction) can be enhanced on personal devices without compromising privacy.
 - Industrial: Assets manufactured by a third party (e.g. printers) can be used to collaborately learn predictive models without each customer having to share its data. For example, data from various assets in different settings can be used to predict maintenance needs or optimize performance without exposing individual usage patterns or sensitive business information.


### Starter Kit outline
This Starter Kit will demonstrate the application of the FedRepo methodology using a real-world dataset. First, the dataset will be described, which contains electricity consumption data of UK households. Then, the FedRepo methodology will be explained and discussed through its key steps, while applying them on a subset of the households. Finally, the performance of the methodology will be evaluated, also in terms of its adaptability and concept drift mitigation. For a more in-depth explanation of all steps involved, please consult the paper.

## Dataset
The forecasting of electricity consumption across households is a highly relevant application for this methodology as energy consumption of households obviously is privacy-sensitive. This was demonstrated in [2], where it is highlighted how household energy patterns can reflect socio-economic statuses. Additionally, many factors could cause for concept drift to occur:

 - The occupation of the household in terms of its inhabitants
 - Replacement of household appliances
 - Fluctuations in electricity prices, encouraging more conservative usage during high price periods
 - ...

The data used is collected by the UK Power Networks led Low Carbon London project ([available here](https://data.london.gov.uk/dataset/smartmeter-energy-use-data-in-london-households)). It consists of 5,567 households (given in column `consumer`) in London representing a balanced sample representative of the Greater London population with a 30-minutes granularity between November 2011 and February 2014. The consumption (in column `consumption`) is given in kWh. For demonstrating our methodology, we randomly selected 300 households for which we ensured that the data is available until at least 01/2014. For these households, a repository of federated models will be trained in order to forecast the consumption within the next 30 minutes. 

In [ ]:
household_subset = sp.get_data(DATA_PATH)
household_subset

## Preprocess data

For ensuring that the data is ready for modelling, we perform the following steps: data cleaning, feature engineering and splitting the data in train and test sets. Features used are the consumption values of up to 6 hours ago (`t-1` being the 30 minutes before the current time, up to `t-12` for the time 6 hours before the current time, in 30-minutes steps), added with the consumption corresponding to same time and 30 minutes before and after on the previous day (`'t-47', 't-48', 't-49'`) and week (`'t-335', 't-336', 't-337'`). Additionally, the day of the week (`dow`), hour of the day (`hod_*`) and month of the year (`moy_*`) are also defined as features, with the latter two being cyclically endcoded (with suffix `_sin`, `_cos`, respectively).

In [ ]:
prep_data = sp.PreprocessConsumption(data=household_subset)
prep_data.preprocess()
prep_data.features_dict['MAC000018']['train'][0].head()

## FedRepo
The FedRepo algorithm, designed to mitigate concept drift in federated learning environments, is structured around several key principles. These principles ensure that the algorithm dynamically adapts to changes in data distributions across different consumers (households), maintaining the efficacy of the deployed models. In our example on electricity consumption forecasting, we use Random Forest (RF) regressor models for the regression task. Note though, that the approach can be adapted to classification tasks by using RF classifiers and an appropriate performance evaluation metric instead. 

Here, we give an overview of the single steps of the FedRepo approach but will elaborate on each step in the remainder of the notebook. 

 - *Local model training*: each consumer trains its own RF regressor model locally. This ensures that sensitive usage data is never shared across consumers or with the central node. Only model parameters like the number of trees or the minimum number of samples required to split an internal node are shared with the central node.
 - *Federated model construction*: at the central node, federated cluster models are constructed aggregating the insights from local models trained by a group of consumers.
 - *Concept drift detection*: the performance of deployed federated models is regularly evaluated at the local level. This ensures that the framework is able to detect concept drift when needed.
 - *Mitigation*: if concept drift is detected, several maintenance steps are taken to mitigate its effects. This could include the retraining of local models with recent data.

These principles are reflected in the main phases of FedRepo, which are: *Initialisation*, *Model training*, *Context-aware inference* and *Dynamic model maintenance*. These are shown in the image below which gives an overview of the methodology. Throughout the methodology, three repositories (hence the name, FedRepo) kept at the central node are continuously maintained and updated to adapt for concept drift:

- $Θ$: a repository of workers, which contains at any moment the workers for which new federated models need to be constructed.
- $Φ$: a repository of global federated random forest models, which contains at any moment the active (deployed) federated models.
- $Γ$: a repository of tree models, which contains at any moment subsets of trees from local RF models of each worker.

Note that a *worker* refers to a consumer in this use case, however in other applications it could be any type of clients/devices. In the following, each of the main phases will be discussed one by one and executed on the UK Power Networks dataset. 

<table><tr><td><img src='media/fedrepo.png' style="width: 1000px" height: auto;><td></tr></table>

### Initialisation
This is a step to formally initialise the repositories. It is performed at the central node. The repository of federated RF models is empty since no RF models have been constructed yet. Analogously, the workers’ repository contains all available workers since for all of them the federated models still need to be constructed, and the repository of tree models is composed of 300 empty sets, one per worker.

In [ ]:
fedrepo = sp.FedRepo(data=prep_data)
fedrepo.initialise()

The following objects show the initial state of the approach: All workers are in one repository (`worker_repo`), while we have no trees in the `tree_repo` nor any active model in the `active_models_repo`. 

In [ ]:
len(fedrepo.worker_repo), len(fedrepo.tree_repo), len(fedrepo.active_models_repo)

### Training

In this first training step, local models are trained and one global cluster model is constructed. 

1. *Local model training*: each worker trains a local RF model to predict the consumption in the next 30 minutes based on the features described above on its training data. To keep training light, a relatively small training set of three months is used (January to March 2012). The local forests are set to contain 100 trees. Each worker then randomly selects a subset of its trees to donate to the central tree repository for this worker.

2. *Global model construction*: an initial global federated model is created by simply aggregating trees trained by the local workers. The global model is also set to contain 100 trees, so again a random selection across all trees available in the tree repository is made. As there are 300 workers and only 100 trees, not all workers will contribute a tree from their local model to the global model.

3. *Global model evaluation*: the global model is shared back with each worker. Locally, each worker evaluates every tree of the global model on its test data, which is defined to be the first month following the training data (April 2012). Performance scores of each tree are used to construct an evaluation feature vector per worker. For the regression model described here for the electricity consumption, the RMSE score is used as performance metric. The outcome of this step is an evaluation matrix, containing 300 feature vectors of length 100. This matrix is collected centrally.

These steps will be performed by running the cell below. Note that the communication contents between the local workers and the central node have been: the locally trained trees (to the central node), the global model (to the local workers) and the performance scores (back to the central node). No local data was shared across workers or with the central node.

In [ ]:
fedrepo.training()

In order to construct federated models, a clustering step is performed in order to identify workers with similar consumption patterns. For each cluster, a specific cluster model is trained. In this way, workers receive a cluster model that is collaboratively built and shared by workers similar to them and different from workers in other clusters. Specifically, the following actions are taken:

4. *Worker clustering*: to derive customized models for a set of similar workers, workers are split into non-overlapping clusters. This is done based on their evaluation vectors, such that workers with similar performance regarding the trees in the global model, i.e. workers with similar usage patterns, are grouped together. A binary particle swarm optimization (PSO) is used as clustering algorithm [3]. This is an extension of original PSO, where every particle is a binary vector representing a certain cluster configuration. The main advantage of this method is that the number of clusters does not need to be defined beforehand, but only a maximal number is set. In our case, we limit the number of clusters to 15, as we observe three main clusters during our experiments and hence, leave enough space for new cluster to evolve. In theory, any other clustering method which does not require the number of clusters to be known could have been applied here.

5. *Cluster model construction*: having grouped the workers, cluster models are constructed in a similar way as the global model was created. Trees available in the tree repository that were donated by workers in a cluster are pooled together and a random selection of 100 trees is made to create the federated cluster model. Evidently, the size of a cluster decides the relative contribution of individual workers to the cluster model. If a cluster only contains 10 workers, it is highly likely that multiple trees of every worker are included in the cluster model.

Running the cell below will execute the binary PSO clustering on the evaluation feature vectors, which consists of an iterative process to find the optimal clustering configuration with regard to a clustering metric (the silhouette score in this case). After each iteration, one particle represents the clustering solution for the workers with the best silhouette score. This value is shown for each iteration in the plots below. If this local optimum does not change for sufficient iterations, the algorithm is stopped and a re-initialisation is done (a new figure is initiated as well). This enables new clustering configurations that were previously not findable. After five re-initialisations, the cluster configuration corresponding to the global optimum is kept. This should also be the best cluster configuration found overall. Both clustering and cluster model creation steps are performed at the central node.

In [ ]:
fedrepo.clustering()

The figure below shows the evaluation vector of each worker, color coded by which cluster the worker belongs to. As mentioned, the values in the evaluation vectors correspond to RMSE scores of trees when applied to a worker's test data. The figure below allows to easily observe the differences between workers with regard to the performance of each tree of the global model. Clustering on these vectors in essence enables to identify common behavioural patterns across groups of workers.

In [ ]:
vis.show_clusters(fedrepo)

### Prediction
All workers receive their federated cluster model from the central device. One way to proceed can be to apply these initial cluster models on the full validation set, which is defined as the data between May 2012 and December 2013. To enable comparison, also the predictions of the global and local models are generated.

In [ ]:
fedrepo.generate_without_maintenance()

Below the RMSE scores of all three model types (local , cluster and global) on the validation set are visualized across all clusters. The cluster models typically demonstrate better performance (hence, lower RMSE value) than the overall global model for almost all workers. This indicates that the customized cluster models are a better fit to predict each worker's electricity consumption. In addition, the performance of the cluster models comes close to that of the local models for many workers, indicating that the cluster models are able to adequately capture the workers' behaviour. Local models usually perform best because they are tailored to individual workers. However, training a local model for each (new) worker is not a scalable solution.

In [ ]:
vis.show_static_performance(fedrepo)

### Concept drift mitigation

Instead of naively applying the initial cluster models without modification, FedRepo proposes a framework to perform dynamic maintenance to mitigate concept drift. The basis of this framework is to calculate the RMSE score between the predicted and observed values after each day. This score is compared to a threshold $δ$, that is derived from the model’s performance on the worker's test set. If the RMSE score is above the threshold for three days in a row, concept drift is said to be identified and several maintenance actions are taken. These involve the activation of the global model as replacement. The global model acts as a baseline, one-fits-all model that is assumed to mitigate the effects of concept drift for a worker that no longer fits its customized model.

More formally, the following steps are conducted:

1. *Global model activation*: the federated global model is activated for the worker in question. This means the cluster model previously deployed for the worker is deactivated. However, trees of this worker (if used during construction of the cluster model) remain part of the cluster model which is still being used by other workers.

2. *Repository update*: the worker that uses the global model is added to $Θ$. During the next retraining phase for creating new cluster models, this worker will take part in it. Also, as the worker no longer uses its cluster model, the *support* of the cluster model is adjusted to reflect this. The *support* of a cluster model always indicates how many workers are currently using the model. It is a relative score, i.e. if 150 out of 300 workers use a certain model, its support is 0.5.

3. *Model support check*: for each federated cluster model, if its support is lower than a predefined threshold $z$, then the model is deactivated for the remaining workers that still used this model. They are added to the workers’ repository to receive new cluster models. In the meantime, the global model is activated for them.

4. *Workers' repository check*: if the amount of workers waiting in $Θ$ is above a predefined threshold $Δ$, retraining is performed.

Retraining entails that the workers present in $Θ$ train new local models on data gathered in the last three months to match the size of the training data in the initial training phase. The global model is updated by randomly sampling 100 trees from the trees available in the tree repositories. Note that these contain trees already present since the initial training phase, as well as newly donated trees by workers that retrained their local models. Clustering is done in the same way as before, by applying PSO on the evaluation vectors derived from the trees in the global model. New cluster models are created, causing every worker to again have an active cluster model and resetting the support of the global model to 0. Note that for the workers not present in $Θ$ before retraining and therefore still have an active cluster model, nothing has changed.

For this data, a cluster model is said to have too low support when it drops below 10 workers, i.e. $z = 0.033$, and retraining is invoked when the workers' repository surpasses 50 workers, i.e. $Δ = 0.16$. Running the cell below will apply the dynamic concept drift mitigation on the validation set. Maintenance steps, such as global model activation or retraining steps, will be logged.

In [ ]:
fedrepo.concept_drift_mitigation()

## Results
Based on the logs printed above, we can see that three retraining events have occurred. This can be observed more visually when plotting the amount of workers in $Θ$ throughout the validation set. The amount builds up three times until the critical number of 45 workers is reached and retraining is invoked. Remember, workers are added to $Θ$ when the performance of a cluster model decreases (concept drift) or the support of a cluster model drops below the threshold. The fact that $Θ$ is not completely empty after these retraining events indicates that cluster models were formed with a too low support from the start.

In [ ]:
vis.show_worker_repo_hist(fedrepo)

Below, the daily RMSE values of one worker (MAC004556) are visualized. These are the values used for evaluation of the cluster model's performance. Both the RMSE scores for the cluster model created after the initial training phase (called the *static* model), and the scores for the active, *dynamic* cluster model are shown. Note that the active cluster model is subject to retraining, and during some time periods the global model was used for prediction. This enables a comparison between a scenario with and without maintenance of the cluster models. We can observe that this worker takes part in 1 retraining event (as indicated by the vertical line). Before this, the RMSE of the dynamic model already deviates from the static model, as the global model was activated after a period of degrading performance. In this case, the global model already has a better prediction accuracy than the deactivated cluster model. After retraining, the new cluster model clearly is better tailored to the current usage patterns as the RMSE drops significantly.

In [ ]:
vis.show_maintenance_example(fedrepo)

The maintenance effect might have a good influence on the prediction power for one worker, but what about the others? The figure below shows the effect for all workers influenced by maintenance. This includes all workers which at some point were part of a retraining event. This is the case for 85 of 300 workers. The actual maintenance effect is quantified by taking the mean difference between the residuals of the static and dynamic scenarios. The residuals of the dynamic scenario are subtracted from the static scenario, which means a positive mean indicates that the residuals in the dynamic scenario are lower *on average*. This is the case for 62 out of 85 workers that underwent maintenance. Below, all prediction improvements are displayed. Although for some workers the maintenance interventions do not improve the prediction power (and actually degrade it), the overall effect is positive.

In [ ]:
vis.show_maintenance_effect(fedrepo)

## Exercise
Now that we have applied the framework once, we can experiment with different choices for some parameters. An interesting exercise would be to observe the effects of different $Δ$ and $z$ thresholds. These, as you have seen, determine when maintenance actions are taken. The current values for these thresholds correspond to 10 ($z$) and 50 ($Δ$) workers. In theory, any value between 0 and 300 can be selected, but given the role of both thresholds, values close to the current ones make more sense. Another interesting parameter is the number of trees in each local or cluster model. This parameter could influence training and inference time, as well as the accuracy of the models. 

In the cell below, a new *FedRepo* instance is created. Custom parameter settings are set in the `custom_params` dictionary. For $z$ and $Δ$, positive values below 100 will be accepted. `n_trees` should be between 30 and 300. You can use the function calls above to apply the framework to the same data; however, make sure to **replace every** `fedrepo` **reference with the custom instance** `custom_fedrepo`. Alternatively, make use of `custom_fedrepo.execute()` below to execute the initialisation, local training, clustering, and drift detection steps all at once. Also, the visualization calls to analyze the results are provided again.

In [ ]:
custom_params = {'z': 10,
                 'delta': 50,
                 'n_trees': 100}

custom_fedrepo = sp.FedRepo(prep_data)
custom_fedrepo.set_parameters(custom_params)

In [ ]:
## Execute FedRepo
# custom_fedrepo.execute()

In [ ]:
# vis.show_static_performance(custom_fedrepo)
# vis.show_worker_repo_hist(custom_fedrepo)
# vis.show_maintenance_effect(custom_fedrepo)

## Conclusion

This Starter Kit proposed a novel approach to forecast electricity consumption in a distributed learning environment, focusing on the detection and mitigation of concept drift. The introduced method, called *FedRepo*, has several advantages for tackling distributed concept drift. First of all, the concept drift detection comes with a very small overhead, as during the regular inference phase not only the actual forecast values are calculated, but additionally the residuals. Based on the residuals, the concept drift is detected locally, while the repositories are updated in the central node. Furthermore, the concept drift is not assumed to happen at the same time across different workers. Instead, each worker deploys a cluster model built on information from similar workers or activates the global model in case the cluster model's performance is degrading before being assigned to a new cluster. Third, the number of models is not predefined and constant, as the binary PSO allows a flexible number of clusters. With this, any flexible change in behaviour is captured. Finally, as the worker clustering is performed on the model performance only, it is not necessary to share the actual patterns of the workers’ data with the central node.

In this notebook, we have applied our methodology to the electricity consumption data of UK households. The results highlight two primary strengths of our approach. Firstly, customizing models through clustering significantly enhances prediction accuracy. We observed that static cluster models consistently outperform a global federated model and, in some cases, even match the performance of local models. Secondly, implementing maintenance interventions, such as global model activation and retraining, has a positive impact on overall performance. This suggests that concept drift is indeed mitigated to some extent by our approach.

One aspect missing in the current implementation is the ability to merge clusters or add workers involved in retraining to an existing cluster. At present, a new clustering is performed solely within the set of workers undergoing maintenance, making this integration impossible. However, enabling this functionality would likely introduce additional communication overhead.

[1] Tsiporkova, E., De Vis, M., Klein, S., Hristoskova, A., & Boeva, V. (2023). Mitigating concept drift in distributed contexts with dynamic repository of federated models. In 2023 IEEE International Conference on Big Data (BigData) (pp. 2690-2699). IEEE.

[2] Beckel, C., Sadamori, L., & Santini, S. (2013). Automatic socio-economic classification of households using electricity consumption data. In Proceedings of the Fourth International Conference on Future Energy Systems (e-Energy '13) (pp. 75–86). Association for Computing Machinery. https://doi.org/10.1145/2487166.2487175

[3] Omran, M. G. H., Salman, A., & Engelbrecht, A. P. (2006). Dynamic clustering using particle swarm optimization with application in image segmentation. Pattern Analysis and Applications, 8, 332–344. https://doi.org/10.1007/s10044-005-0015-5

©, 2024, Sirris